In [37]:
import pandas as pd
import numpy as np

Исходные данные 

уровень значимости (вероятность ошибки 1-го рода, т.е. отвергнуть Н0 когда она верна)

In [39]:
alpha = 0.05

мощность (вероятность ошибки при отклонении альтернативной гипотезы — по умолчанию, в научных работах используется 80%: это означает, что в 1 из 5 случаев мы не найдем разницы, если она, на самом деле, существует))

In [40]:
n = 0.8

текущее значение конверсии

In [41]:
current_conversion = 0.2

желаемый эффект

In [42]:
mde = 0.05

Посчитаем минимальный размер выборки

In [33]:
#табличное значение для мощности 80%, 85%, 90%, 95%
zb_08 = 0.84
zb_085 = 1.04
zb_09 = 1.28
zb_095 = 1.64

#табличное значение для уровня значимости в 5%, 1% односторонняя гипотеза 
za1_005 = 1.65
za1_001 = 2.33

#табличное значение для уровня значимости в 5%, 1% двухсторонняя гипотеза
za2_005 = 1.96
za2_001 = 2.58

In [38]:
sample_size = round((2*((za2_005 + zb)**2)*current_conversion*(1 - current_conversion))/mde**2)
print("Минимальный размер одной группы:", sample_size)

Минимальный размер одной группы: 1004


Смоделируем выборки  
a - выборка на продукте без изменений   
b - выборка на продукте с изменениями (предположим, что желаемый эффект достигнут)  

In [43]:
a = np.random.binomial(1, current_conversion, size=sample_size)

b = np.random.binomial(1, current_conversion + mde, size=sample_size)

Посмотрим на размеры конверсий 

In [62]:
conversion_a = a.mean()
print("Конверсия в выборке А составила: {:.2f}".format(conversion_a))

Конверсия в выборке А составила: 0.21


In [63]:
conversion_b = b.mean()
print("Конверсия в выборке B составила: {:.2f}".format(conversion_b))

Конверсия в выборке B составила: 0.26


Для оценки стат. значимости используем z-test (Chi-square)

In [56]:
from statsmodels.stats.proportion import proportions_ztest

In [60]:
pvalue = proportions_ztest([a.sum(), b.sum()], [len(a), len(b)])

In [64]:
print("Значение p_value: {:.3f}".format(pvalue[1]))

Значение p_value: 0.013


Получив такое значение p-value можно отклонить Н0. Т.е. подтвердить, что конверсии различаются стат. значимо. (Что логично т.к. мы искуственно создали прирост в 5%)

## Bootstrap

Смоделируем проведение нашего эксперимента 10000 раз

In [90]:
pvalue_lst = []

for i in range(10000):
    a = np.random.binomial(1, current_conversion, size=sample_size)
    b = np.random.binomial(1, current_conversion + mde, size=sample_size)
    pvalue = proportions_ztest([a.sum(), b.sum()], [len(a), len(b)])
    pvalue_lst.append(pvalue[1])

In [91]:
_ = (pd.Series(pvalue_lst) < 0.05).mean()

In [92]:
print("В {:.2%} случаях мы получили стат. значимое различие конверсий".format(_))

В 76.93% случаях мы получили стат. значимое различие конверсий


Несмотря на то, что мы точно знаем, что конверсия в группе B отличается на необходимую нам величину (т.к. выборки создаются искусственно). При проведении теста 10000 раз, выявились стат.значимые различия в 77% случаев (что приблизительно соответствует заданной мощности в 80%, для более точного схождения необходимо провести больше экспериментов)

Проверим, подходит ли Z-test для сравнения конверсий  
Для этого проведем АА тест 10000 раз.

In [85]:
pvalue_lst = []

for i in range(10000):
    a = np.random.binomial(1, current_conversion, size=sample_size)
    b = np.random.binomial(1, current_conversion, size=sample_size)
    i, pvalue = proportions_ztest([a.sum(), b.sum()], [len(a), len(b)])
    pvalue_lst.append(pvalue)

In [88]:
_ = (pd.Series(pvalue_lst) < 0.05).mean()

0.0515

Видим, что приблизительно в 5% случаев мы отклоняем Н0, хотя конверсии фактически не отличаются. Можно сделалть вывод, что можно использовать Z-test для проверки стат. значимого отличия конверсий. Т.к. полученный уровень значимости совпадает с тем, что мы задали изначально